In [1]:
#pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0
#pip install keras-image-helper

In [2]:
import  grpc

import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

from keras_image_helper import create_preprocessor

2022-11-28 21:31:57.743432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#where our tf-serving service is running. running in localhost inside docker
host= 'localhost:8500'

#access the previous port, since we are running things locally we can use insecure channel
#since it is running in kubernetes (gateway and tf-serving) tf-serving will not be accessible
#from the outside
channel = grpc.insecure_channel(host)

# we will use this channel to connect our predictions service. This prediction service, 
#is nothing more that tf_serving

#invoke remote services, we will be using for communicating with the actual service. 
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [4]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [5]:
url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [6]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [7]:
# now we want to send X to prediction service that is running in tf-serving
# we also need to prepare the request, is not JSOn format

pb_request = predict_pb2.PredictRequest()

pb_request.model_spec.name='clothing-model'
pb_request.model_spec.signature_name='serving_default'
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

In [8]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [13]:
preds = pb_response.outputs['dense_7'].float_val

In [12]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
] 

In [14]:
dict(zip(classes, preds))

{'dress': -1.8682901859283447,
 'hat': -4.761244773864746,
 'longsleeve': -2.316983461380005,
 'outwear': -1.062570333480835,
 'pants': 9.88715934753418,
 'shirt': -2.8124334812164307,
 'shoes': -3.666282892227173,
 'shorts': 3.200361490249634,
 'skirt': -2.6023383140563965,
 't-shirt': -4.835045337677002}